# Setting Up the Environment

In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/efficientnet-1.0.0-py37h06a4308_0.tar.bz2' -y
!conda install '/kaggle/input/pydicom-conda-helper/keras-applications-1.0.8-py_1.tar.bz2' -c conda-forge -y

# Making Necessary Directories and Setting Paths

In [ ]:
!mkdir -p ./test_images_rgb/
!mkdir -p ./test_images_psuedo_color/
!mkdir -p ./model_1_outputs/
!mkdir -p ./model_2_outputs/
!mkdir -p ./model_3_outputs/
!mkdir -p ./model_4_outputs/
!mkdir -p ./model_5_outputs/
!mkdir -p ./model_6_outputs/


rgb_test_images_path = './test_images_rgb/'
psuedo_color_test_images_path = './test_images_psuedo_color/'
model_1_outputs_path = './model_1_outputs/'
model_2_outputs_path = './model_2_outputs/'
model_3_outputs_path = './model_3_outputs/'
model_4_outputs_path = './model_4_outputs/'
model_5_outputs_path = './model_5_outputs/'
model_6_outputs_path = './model_6_outputs/'

print(rgb_test_images_path)
print(psuedo_color_test_images_path)
print(model_1_outputs_path)
print(model_2_outputs_path)
print(model_3_outputs_path)
print(model_4_outputs_path)
print(model_5_outputs_path)
print(model_6_outputs_path)
print('Done')

In [ ]:
image_height_width = {}

image_four_class_classification_model_1, image_four_class_classification_model_2, image_four_class_classification_model_3 = {}, {}, {}
image_four_class_classification_model_4, image_four_class_classification_model_5 = {}, {}

image_four_class_classification_model_6, image_four_class_classification_model_7, image_four_class_classification_model_8 = {}, {}, {}
image_four_class_classification_model_9, image_four_class_classification_model_10 = {}, {}
print('Done')

# Imports

In [ ]:
import tensorflow as tf
print(tf.__version__)

import os
import gc
import cv2
import glob
import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map, thread_map
import copy


import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

from skimage.filters import unsharp_mask
from skimage import exposure

import albumentations as albu

import tensorflow_hub as tf_hub

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
print('Done')

# Functions

In [ ]:
def get_detection_data(input_size=(608, 608), original_size=None, model_outputs=None, class_names=None,
                       display_num=False):
    """
    This function takes the output of the YOLOv4 model and deciphers the detections returning a
    dataframe with the following values: x_min, y_min, x_max, y_max, class_name, confidence_score, bounding_box_width,
    bounding_box_height
    :param input_size: input size of the image to the YOLO model
    :param original_size: tuple containing the size of the actual image
    :param model_outputs: outputs from inference_model
    :param class_names: list of object class names
    :param display_num: Boolean variable to display the number of boxes
    :return: Pandas dataframe with all the values
    """

    num_bboxes = model_outputs[-1][0]
    boxes, scores, classes = [output[0][:num_bboxes] for output in model_outputs[:-1]]

    h, w = input_size[:2]
    df = pd.DataFrame(boxes, columns=['x1', 'y1', 'x2', 'y2'])
    df.insert(0, 'image_width', original_size[1])
    df.insert(1, 'image_height', original_size[0])
    df[['x1', 'x2']] = (df[['x1', 'x2']] * w).astype('int64')
    df[['y1', 'y2']] = (df[['y1', 'y2']] * h).astype('int64')
    df['class_name'] = np.array(class_names)[classes.astype('int64')]
    df['score'] = scores
    df[['x1', 'x2']] = ((df[['x1', 'x2']].divide(input_size[1])).
                        multiply(df['image_width'], axis='index')).astype('int64')
    df[['y1', 'y2']] = ((df[['y1', 'y2']].divide(input_size[0])).
                        multiply(df['image_height'], axis='index')).astype('int64')
    df['bounding_box_width'] = df['x2'] - df['x1']
    df['bounding_box_height'] = df['y2'] - df['y1']

    if display_num:
        print(f'# of bboxes: {num_bboxes}')

    return df

In [ ]:
# The submisison requires xmin, ymin, xmax, ymax format. 
# YOLOv5 returns x_center, y_center, width, height
def correct_bbox_format(bboxes, H, W):
    correct_bboxes = []
    
    for b in bboxes:
        xc, yc = int(np.round(b[0] * W)), int(np.round(b[1] * H))
        w, h = int(np.round(b[2] * W)), int(np.round(b[3] * H))

        xmin = xc - int(np.round(w/2))
        xmax = xc + int(np.round(w/2))
        ymin = yc - int(np.round(h/2))
        ymax = yc + int(np.round(h/2))
        
        correct_bboxes.append([xmin, ymin, xmax, ymax])
        
    return correct_bboxes

# Read the txt file generated by YOLOv5 during inference and extract 
# confidence and bounding box coordinates.
def get_conf_bboxes(file_path):
    confidence = []
    bboxes = []
    with open(file_path, 'r') as file:
        for line in file:
            #print(line)
            preds = line.strip('\n').split(' ')
            #print(preds)
            preds = list(map(float, preds))
            #print(preds)
            confidence.append(preds[-1])
            #print(confidence)
            bboxes.append(preds[1:-1])
            #print(bboxes)
    return confidence, bboxes

In [ ]:
def reduce_bounding_box_size(image_name,annotation, percentage):
    """
    Reduces the bounding box size by a fixed percentage
    """
    box_width = annotation[4] - annotation[2]
    box_height = annotation[5] - annotation[3]
    
    box_width_delta = ((percentage/2)/100)*box_width
    box_height_delta = ((percentage/2)/100)*box_height
    
    annotation[2], annotation[4] = int(annotation[2] - box_width_delta), int(annotation[4] + box_width_delta)
    annotation[3], annotation[5] = int(annotation[3] - box_height_delta), int(annotation[5] + box_height_delta)
    
    H, W = image_height_width[image_name][0],image_height_width[image_name][1]
    
    if annotation[2] < 0:
        annotation[2] = 0
    if annotation[3] < 0:
        annotation[3] = 0
    if annotation[4] > W:
        annotation[4] = W
    if annotation[5] > H:
        annotation[5] = H
    
    return annotation

In [ ]:
def bounding_box_sorter(image_name,prediction_string):
    prediction_string_parts = prediction_string.split(' ')
    all_predictions = []
    for i in range(0,len(prediction_string_parts), 6):
        temp = prediction_string_parts[i:i+6]
        #print(temp)
        temp[1], temp[2], temp[3], temp[4], temp[5] = float(temp[1]), int(temp[2]), int(temp[3]), int(temp[4]), int(temp[5])
        #temp = reduce_bounding_box_size(image_name,temp, 1)
        all_predictions.append(temp)
    
    #print(all_predictions)
    all_predictions.sort(key = lambda x: x[1])
    #print(all_predictions)
    all_predictions = all_predictions[::-1]
    #print(all_predictions)
    
    prediction_string = ''
    for bbox in all_predictions:
        prediction_string += bbox[0] +' ' + str(bbox[1]) + ' ' + str(bbox[2]) +' ' + str(bbox[3]) +' ' + str(bbox[4]) +' ' + str(bbox[5]) + ' '
    
    return prediction_string[:-1]

In [ ]:
def confidence_score_calibrator(prediction_string, class_confidence):
    prediction_string_parts = prediction_string.split(' ')
    all_predictions = []
    for i in range(0,len(prediction_string_parts), 6):
        temp = prediction_string_parts[i:i+6]
        #print(temp)
        temp[1], temp[2], temp[3], temp[4], temp[5] = float(temp[1]), int(temp[2]), int(temp[3]), int(temp[4]), int(temp[5]) 
        all_predictions.append(temp)
    
    none_prediction = all_predictions[-1]
    all_predictions = all_predictions[:-1]
    
    #print(class_confidence)
    class_confidence_value = class_confidence[np.argmax(class_confidence)]
    #print(class_confidence_value)
    for value in all_predictions:
        value[1] += value[1]-(value[1]*pow(class_confidence_value, 0.1)*pow(none_prediction[1], 0.1))
        
    prediction_string = ''
    for bbox in all_predictions:
        prediction_string += bbox[0] +' ' + str(bbox[1]) + ' ' + str(bbox[2]) +' ' + str(bbox[3]) +' ' + str(bbox[4]) +' ' + str(bbox[5]) + ' '
        
    prediction_srting = bounding_box_sorter(prediction_string[:-1])
    
    prediction_string += none_prediction[0] +' ' + str(none_prediction[1]) + ' ' + str(none_prediction[2]) +' ' + str(none_prediction[3]) +' ' + str(none_prediction[4]) +' ' + str(none_prediction[5]) + ' '
    
    return prediction_string[:-1]

In [ ]:
def keep_n_top_loaclisations(prediction_string, n):
    prediction_string_parts = prediction_string.split(' ')
    total_parts = n*6
    
    shorter_prediction_string = ''
    for i in range(0,total_parts):
        shorter_prediction_string += prediction_string_parts[i] + ' '
    
    shorter_prediction_string += prediction_string_parts[-6] + ' ' + prediction_string_parts[-5] + ' ' + prediction_string_parts[-4] + ' ' + prediction_string_parts[-3] + ' ' + prediction_string_parts[-2] + ' ' + prediction_string_parts[-1]
    
    return shorter_prediction_string

In [ ]:
def denormalise(bboxes, H, W):
    """
    This function converts the normalised bounding boxes in pascal format to denormalised bounding boxes
    :param bboxes:
    :param H:
    :param W:
    :return:
    """
    for box in bboxes:
        box[0], box[1], box[2], box[3] = int(box[0]*W), int(box[1]*H), int(box[2]*W), int(box[3]*H)

    return bboxes

In [ ]:
def bb_intersection_over_union(A, B) -> float:
    xA = max(A[0], B[0])
    yA = max(A[1], B[1])
    xB = min(A[2], B[2])
    yB = min(A[3], B[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)

    if interArea == 0:
        return 0.0

    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (A[2] - A[0]) * (A[3] - A[1])
    boxBArea = (B[2] - B[0]) * (B[3] - B[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [ ]:
def prefilter_boxes(boxes, scores, labels, weights, thr):
    # Create dict with boxes stored by its label
    new_boxes = dict()

    for t in range(len(boxes)):

        if len(boxes[t]) != len(scores[t]):
            print('Error. Length of boxes arrays not equal to length of scores array: {} != {}'.format(len(boxes[t]), len(scores[t])))
            exit()

        if len(boxes[t]) != len(labels[t]):
            print('Error. Length of boxes arrays not equal to length of labels array: {} != {}'.format(len(boxes[t]), len(labels[t])))
            exit()

        for j in range(len(boxes[t])):
            score = scores[t][j]
            if score < thr:
                continue
            label = int(labels[t][j])
            box_part = boxes[t][j]
            x1 = float(box_part[0])
            y1 = float(box_part[1])
            x2 = float(box_part[2])
            y2 = float(box_part[3])

            # Box data checks
            if x2 < x1:
                warnings.warn('X2 < X1 value in box. Swap them.')
                x1, x2 = x2, x1
            if y2 < y1:
                warnings.warn('Y2 < Y1 value in box. Swap them.')
                y1, y2 = y2, y1
            if x1 < 0:
                warnings.warn('X1 < 0 in box. Set it to 0.')
                x1 = 0
            if x1 > 1:
                warnings.warn('X1 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                x1 = 1
            if x2 < 0:
                warnings.warn('X2 < 0 in box. Set it to 0.')
                x2 = 0
            if x2 > 1:
                warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                x2 = 1
            if y1 < 0:
                warnings.warn('Y1 < 0 in box. Set it to 0.')
                y1 = 0
            if y1 > 1:
                warnings.warn('Y1 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                y1 = 1
            if y2 < 0:
                warnings.warn('Y2 < 0 in box. Set it to 0.')
                y2 = 0
            if y2 > 1:
                warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                y2 = 1
            if (x2 - x1) * (y2 - y1) == 0.0:
                warnings.warn("Zero area box skipped: {}.".format(box_part))
                continue

            # [label, score, weight, model index, x1, y1, x2, y2]
            b = [int(label), float(score) * weights[t], weights[t], t, x1, y1, x2, y2]
            if label not in new_boxes:
                new_boxes[label] = []
            new_boxes[label].append(b)

    # Sort each list in dict by score and transform it to numpy array
    for k in new_boxes:
        current_boxes = np.array(new_boxes[k])
        new_boxes[k] = current_boxes[current_boxes[:, 1].argsort()[::-1]]

    return new_boxes

In [ ]:
def get_weighted_box(boxes, conf_type='avg'):
    """
    Create weighted box for set of boxes
    :param boxes: set of boxes to fuse
    :param conf_type: type of confidence one of 'avg' or 'max'
    :return: weighted box (label, score, weight, x1, y1, x2, y2)
    """

    box = np.zeros(8, dtype=np.float32)
    conf = 0
    conf_list = []
    w = 0
    for b in boxes:
        box[4:] += (b[1] * b[4:])
        conf += b[1]
        conf_list.append(b[1])
        w += b[2]
    box[0] = boxes[0][0]
    if conf_type == 'avg':
        box[1] = conf / len(boxes)
    elif conf_type == 'max':
        box[1] = np.array(conf_list).max()
    elif conf_type in ['box_and_model_avg', 'absent_model_aware_avg']:
        box[1] = conf / len(boxes)
    box[2] = w
    box[3] = -1 # model index field is retained for consistensy but is not used.
    box[4:] /= conf
    return box

In [ ]:
def find_matching_box(boxes_list, new_box, match_iou):
    best_iou = match_iou
    best_index = -1
    for i in range(len(boxes_list)):
        box = boxes_list[i]
        if box[0] != new_box[0]:
            continue
        iou = bb_intersection_over_union(box[4:], new_box[4:])
        if iou > best_iou:
            best_index = i
            best_iou = iou

    return best_index, best_iou

In [ ]:
def weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=None, iou_thr=0.55, skip_box_thr=0.0, conf_type='box_and_model_avg', allows_overflow=False):
    '''
    :param boxes_list: list of boxes predictions from each model, each box is 4 numbers.
    It has 3 dimensions (models_number, model_preds, 4)
    Order of boxes: x1, y1, x2, y2. We expect float normalized coordinates [0; 1]
    :param scores_list: list of scores for each model
    :param labels_list: list of labels for each model
    :param weights: list of weights for each model. Default: None, which means weight == 1 for each model
    :param iou_thr: IoU value for boxes to be a match
    :param skip_box_thr: exclude boxes with score lower than this variable
    :param conf_type: how to calculate confidence in weighted boxes. 'avg': average value, 'max': maximum value, 'box_and_model_avg': box and model wise hybrid weighted average, 'absent_model_aware_avg': weighted average that takes into account the absent model.
    :param allows_overflow: false if we want confidence score not exceed 1.0
    :return: boxes: boxes coordinates (Order of boxes: x1, y1, x2, y2).
    :return: scores: confidence scores
    :return: labels: boxes labels
    '''

    if weights is None:
        weights = np.ones(len(boxes_list))
    if len(weights) != len(boxes_list):
        print('Warning: incorrect number of weights {}. Must be: {}. Set weights equal to 1.'.format(len(weights), len(boxes_list)))
        weights = np.ones(len(boxes_list))
    weights = np.array(weights)

    if conf_type not in ['avg', 'max', 'box_and_model_avg', 'absent_model_aware_avg']:
        print('Unknown conf_type: {}. Must be "avg", "max" or "box_and_model_avg", or "absent_model_aware_avg"'.format(conf_type))
        exit()

    filtered_boxes = prefilter_boxes(boxes_list, scores_list, labels_list, weights, skip_box_thr)
    if len(filtered_boxes) == 0:
        return np.zeros((0, 4)), np.zeros((0,)), np.zeros((0,))

    overall_boxes = []
    for label in filtered_boxes:
        boxes = filtered_boxes[label]
        new_boxes = []
        weighted_boxes = []
        # Clusterize boxes
        for j in range(0, len(boxes)):
            index, best_iou = find_matching_box(weighted_boxes, boxes[j], iou_thr)
            if index != -1:
                new_boxes[index].append(boxes[j])
                weighted_boxes[index] = get_weighted_box(new_boxes[index], conf_type)
            else:
                new_boxes.append([boxes[j].copy()])
                weighted_boxes.append(boxes[j].copy())
        # Rescale confidence based on number of models and boxes
        for i in range(len(new_boxes)):
            clustered_boxes = np.array(new_boxes[i])
            if conf_type == 'box_and_model_avg':
                # weighted average for boxes
                weighted_boxes[i][1] = weighted_boxes[i][1] * len(clustered_boxes) / weighted_boxes[i][2]
                # identify unique model index by model index column
                _, idx = np.unique(clustered_boxes[:, 3], return_index=True)
                # rescale by unique model weights
                weighted_boxes[i][1] = weighted_boxes[i][1] *  clustered_boxes[idx, 2].sum() / weights.sum()

            elif conf_type == 'absent_model_aware_avg':
                # get unique model index in the cluster
                models = np.unique(clustered_boxes[:, 3]).astype(int)
                # create a mask to get unused model weights
                mask = np.ones(len(weights), dtype=bool)
                mask[models] = False
                # absent model aware weighted average
                weighted_boxes[i][1] = weighted_boxes[i][1] * len(clustered_boxes) / (weighted_boxes[i][2] + weights[mask].sum())
            elif not allows_overflow:
                weighted_boxes[i][1] = weighted_boxes[i][1] * min(weights.sum(), len(clustered_boxes)) / weights.sum()
            else:
                weighted_boxes[i][1] = weighted_boxes[i][1] * len(clustered_boxes) / weights.sum()
        overall_boxes.append(np.array(weighted_boxes))
    overall_boxes = np.concatenate(overall_boxes, axis=0)
    overall_boxes = overall_boxes[overall_boxes[:, 1].argsort()[::-1]]
    boxes = overall_boxes[:, 4:]
    scores = overall_boxes[:, 1]
    labels = overall_boxes[:, 0]
    return boxes, scores, labels

In [ ]:
def image_extraction_and_saving(path):
    global image_height_width
    my_dicom = pydicom.dcmread(path)

    image_data = apply_voi_lut(my_dicom.pixel_array, my_dicom)

    if my_dicom.PhotometricInterpretation == 'MONOCHROME1':
        image_data = np.amax(image_data) - image_data

    image_data = image_data - np.min(image_data)
    image_data = image_data / np.max(image_data)
    image_data = (image_data*255).astype(np.uint8)


    my_shape = np.shape(image_data)
    file_parts = path.split('/')
    image_height_width[file_parts[-1].split('.')[0]+'_image'] = my_shape
    cv2.imwrite(os.path.join(rgb_test_images_path, file_parts[-3]+'_'+file_parts[-2]+'_'+file_parts[-1].split('.')[0]+'.jpeg'), np.stack((cv2.resize(image_data, (1280,1280)),)*3, axis=-1))
    cv2.imwrite(os.path.join(psuedo_color_test_images_path, file_parts[-3]+'_'+file_parts[-2]+'_'+file_parts[-1].split('.')[0]+'.jpeg'), cv2.applyColorMap(np.stack((cv2.resize(image_data, (1280,1280)),)*3, axis=-1), cv2.COLORMAP_JET))


# Extracting Images

In [ ]:
all_files = glob.glob('../input/siim-covid19-detection/test' + '/**/*.dcm', recursive=True)

if len(pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')) == 2477:
    all_files = all_files[:10]

In [ ]:
_ = thread_map(image_extraction_and_saving, all_files, max_workers=8, chunksize=1)
print('All files extracted')

In [ ]:
gc.collect()

In [ ]:
all_files = os.listdir(rgb_test_images_path)

# Custom Class for 4-Class Classification Models

In [ ]:
MODEL_ARCH = 'efficientnetv2-l-21k-ft1k'
# Get the TensorFlow Hub model URL
hub_type = 'feature_vector' # ['classification', 'feature_vector']
MODEL_ARCH_PATH = f'../input/efficientnetv2-tfhub-weight-files/tfhub_models/{MODEL_ARCH}/{hub_type}'

# Custom wrapper class to load the right pretrained weights explicitly from the local directory
class KerasLayerWrapper(tf_hub.KerasLayer):
    def __init__(self, handle, **kwargs):
        handle = tf_hub.KerasLayer(tf_hub.load(MODEL_ARCH_PATH))
        super().__init__(handle, **kwargs)
        
print('Done')

# **Study Level (4 Class) Classification**

In [ ]:
my_submission_file_1_study = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
my_submission_file_2_study = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
my_submission_file_3_study = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
my_submission_file_4_study = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
my_submission_file_5_study = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')

image_size_1 = (768, 768, 3)
image_size_2 = (768, 768, 3)
image_size_3 = (768, 768, 3)
image_size_4 = (768, 768, 3)
image_size_5 = (768, 768, 3)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[0],
                 [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=13000)])
strategy = tf.distribute.OneDeviceStrategy(device='/gpu:0')

my_model_1 = tf.keras.models.load_model('../input/d/asadmkhan/siim-classification-models/model0_ver15.h5')
my_model_1 = tf.keras.Model(inputs=my_model_1.input, outputs=my_model_1.get_layer('Classification_Head').output)
print('Model 1 Loaded')

study_level = my_submission_file_1_study[my_submission_file_1_study['id'].str.contains('_study')]
print(study_level.head())
print(len(study_level))

In [ ]:
for file in tqdm(range(len(study_level))):
    my_row = study_level.loc[file].values.tolist()
    study_name = my_row[0].split('_')

    complete_name = [x for x in all_files if study_name[0] in x]
    
    my_predictions_study = np.zeros((1,4))
    for images in complete_name:
        #print(images)
        
        image_data = cv2.imread(os.path.join(rgb_test_images_path, images), -1)
        if image_data is not None:
            my_image_1 = cv2.resize(image_data, image_size_1[:2])/255.0
            my_image_expanded_1 = np.zeros((3,*image_size_1))
            my_image_expanded_1[0,...] = my_image_1
            my_image_expanded_1[1,...] = np.fliplr(my_image_1)
            my_image_expanded_1[2,...] = np.roll(my_image_1, 30, axis=1)
            
            my_predictions_1 = my_model_1.predict(my_image_expanded_1)
            
            #try:
            #    index = np.argmax(my_predictions_1[:,statistics.mode(np.argmax(my_predictions_1, axis=0))],axis=0)
            #    my_predictions_1 = np.reshape(my_predictions_1[index,:], (1,4))
            #except:
            my_predictions_1 = np.reshape(np.mean(my_predictions_1, axis=0), (1,4))
            
            
            del my_image_1
            del my_image_expanded_1
            
            my_predictions_study = np.concatenate((my_predictions_study, my_predictions_1))
            
            #my_prediction_image = np.mean(np.vstack((my_predictions_1)), axis=0)
            image_four_class_classification_model_1[images.split('_')[-1].split('.')[0]+'_image'] = my_predictions_1
            
            
            
    #print(my_predictions_study)
    if len(my_predictions_study) > 1:
        my_predictions_study = list(my_predictions_study[1,:])


        prediction_string = 'negative' + ' ' + str(my_predictions_study[0]) + ' 0 0 1 1 ' + 'typical' + ' ' + str(my_predictions_study[1]) + ' 0 0 1 1 ' + 'indeterminate' + ' ' + str(my_predictions_study[2]) + ' 0 0 1 1 ' + 'atypical' + ' ' + str(my_predictions_study[3]) + ' 0 0 1 1' 

        #print(prediction_string)

        search_name = study_name[0] + '_study'
        my_submission_file_1_study['PredictionString'][my_submission_file_1_study.id==search_name] = prediction_string

In [ ]:
my_submission_file_1_study

In [ ]:
del my_model_1
gc.collect()
my_submission_file_1_study

In [ ]:
my_model_2 = tf.keras.models.load_model('../input/d/asadmkhan/siim-classification-models/model1_ver15.h5')
my_model_2 = tf.keras.Model(inputs=my_model_2.input, outputs=my_model_2.get_layer('Classification_Head').output)

print('Model 2 Loaded')

study_level = my_submission_file_2_study[my_submission_file_2_study['id'].str.contains('_study')]
print(study_level.head())
print(len(study_level))

In [ ]:
for file in tqdm(range(len(study_level))):
    my_row = study_level.loc[file].values.tolist()
    study_name = my_row[0].split('_')

    complete_name = [x for x in all_files if study_name[0] in x]
    
    my_predictions_study = np.zeros((1,4))
    for images in complete_name:
        #print(images)
        
        image_data = cv2.imread(os.path.join(rgb_test_images_path, images), -1)
        if image_data is not None:
            my_image_2 = cv2.resize(image_data, image_size_2[:2])/255.0
            my_image_expanded_2 = np.zeros((3,*image_size_2))
            my_image_expanded_2[0,...] = my_image_2
            my_image_expanded_2[1,...] = np.fliplr(my_image_2)
            my_image_expanded_2[2,...] = np.roll(my_image_2, 30, axis=1)
            
            my_predictions_2 = my_model_2.predict(my_image_expanded_2)
            
            #try:
            #    index = np.argmax(my_predictions_2[:,statistics.mode(np.argmax(my_predictions_2, axis=0))],axis=0)
            #    my_predictions_2 = np.reshape(my_predictions_2[index,:], (1,4))
            #except:
            my_predictions_2 = np.reshape(np.mean(my_predictions_2, axis=0), (1,4))
            
            
            del my_image_2
            del my_image_expanded_2
            
            my_predictions_study = np.concatenate((my_predictions_study, my_predictions_2))
            
            #my_prediction_image = np.mean(np.vstack((my_predictions_1)), axis=0)
            image_four_class_classification_model_2[images.split('_')[-1].split('.')[0]+'_image'] = my_predictions_2
            
            
    #print(my_predictions_study)
    if len(my_predictions_study) > 1:
        my_predictions_study = list(my_predictions_study[1,:])


        prediction_string = 'negative' + ' ' + str(my_predictions_study[0]) + ' 0 0 1 1 ' + 'typical' + ' ' + str(my_predictions_study[1]) + ' 0 0 1 1 ' + 'indeterminate' + ' ' + str(my_predictions_study[2]) + ' 0 0 1 1 ' + 'atypical' + ' ' + str(my_predictions_study[3]) + ' 0 0 1 1' 

        #print(prediction_string)

        search_name = study_name[0] + '_study'
        my_submission_file_2_study['PredictionString'][my_submission_file_2_study.id==search_name] = prediction_string

In [ ]:
del my_model_2
gc.collect()

my_submission_file_2_study

In [ ]:
my_model_3 = tf.keras.models.load_model('../input/d/asadmkhan/siim-classification-models/model2_ver15.h5')
my_model_3 = tf.keras.Model(inputs=my_model_3.input, outputs=my_model_3.get_layer('Classification_Head').output)
print('Model 3 Loaded')

study_level = my_submission_file_3_study[my_submission_file_3_study['id'].str.contains('_study')]
print(study_level.head())
print(len(study_level))

In [ ]:
for file in tqdm(range(len(study_level))):
    my_row = study_level.loc[file].values.tolist()
    study_name = my_row[0].split('_')

    complete_name = [x for x in all_files if study_name[0] in x]
    
    my_predictions_study = np.zeros((1,4))
    for images in complete_name:
        #print(images)
        
        image_data = cv2.imread(os.path.join(rgb_test_images_path, images), -1)
        if image_data is not None:
            my_image_3 = cv2.resize(image_data, image_size_3[:2])/255.0
            my_image_expanded_3 = np.zeros((3,*image_size_3))
            my_image_expanded_3[0,...] = my_image_3
            my_image_expanded_3[1,...] = np.fliplr(my_image_3)
            my_image_expanded_3[2,...] = np.roll(my_image_3, 30, axis=1)
            
            my_predictions_3 = my_model_3.predict(my_image_expanded_3)
            
            #try:
            #    index = np.argmax(my_predictions_3[:,statistics.mode(np.argmax(my_predictions_3, axis=0))],axis=0)
            #    my_predictions_3 = np.reshape(my_predictions_3[index,:], (1,4))
            #except:
            my_predictions_3 = np.reshape(np.mean(my_predictions_3, axis=0), (1,4))
            
            del my_image_3
            del my_image_expanded_3
            
            my_predictions_study = np.concatenate((my_predictions_study, my_predictions_3))
            
            #my_prediction_image = np.mean(np.vstack((my_predictions_1)), axis=0)
            image_four_class_classification_model_3[images.split('_')[-1].split('.')[0]+'_image'] = my_predictions_3
            
            
    #print(my_predictions_study)
    if len(my_predictions_study) > 1:
        my_predictions_study = list(my_predictions_study[1,:])


        prediction_string = 'negative' + ' ' + str(my_predictions_study[0]) + ' 0 0 1 1 ' + 'typical' + ' ' + str(my_predictions_study[1]) + ' 0 0 1 1 ' + 'indeterminate' + ' ' + str(my_predictions_study[2]) + ' 0 0 1 1 ' + 'atypical' + ' ' + str(my_predictions_study[3]) + ' 0 0 1 1' 

        #print(prediction_string)

        search_name = study_name[0] + '_study'
        my_submission_file_3_study['PredictionString'][my_submission_file_3_study.id==search_name] = prediction_string

In [ ]:
del my_model_3
gc.collect()

my_submission_file_3_study

In [ ]:
my_model_4 = tf.keras.models.load_model('../input/d/asadmkhan/siim-classification-models/model3_ver15.h5')
my_model_4 = tf.keras.Model(inputs=my_model_4.input, outputs=my_model_4.get_layer('Classification_Head').output)
print('Model 4 Loaded')

study_level = my_submission_file_4_study[my_submission_file_4_study['id'].str.contains('_study')]
print(study_level.head())
print(len(study_level))

In [ ]:
for file in tqdm(range(len(study_level))):
    my_row = study_level.loc[file].values.tolist()
    study_name = my_row[0].split('_')

    complete_name = [x for x in all_files if study_name[0] in x]
    
    my_predictions_study = np.zeros((1,4))
    for images in complete_name:
        #print(images)
        
        image_data = cv2.imread(os.path.join(rgb_test_images_path, images), -1)
        if image_data is not None:
            my_image_4 = cv2.resize(image_data, image_size_4[:2])/255.0
            my_image_expanded_4 = np.zeros((3,*image_size_4))
            my_image_expanded_4[0,...] = my_image_4
            my_image_expanded_4[1,...] = np.fliplr(my_image_4)
            my_image_expanded_4[2,...] = np.roll(my_image_4, 30, axis=1)
            
            my_predictions_4 = my_model_4.predict(my_image_expanded_4)
            
            #try:
            #    index = np.argmax(my_predictions_4[:,statistics.mode(np.argmax(my_predictions_4, axis=0))],axis=0)
            #    my_predictions_4 = np.reshape(my_predictions_4[index,:], (1,4))
            #except:
            my_predictions_4 = np.reshape(np.mean(my_predictions_4, axis=0), (1,4))
            
            del my_image_4
            del my_image_expanded_4
            
            my_predictions_study = np.concatenate((my_predictions_study, my_predictions_4))
            
            #my_prediction_image = np.mean(np.vstack((my_predictions_1)), axis=0)
            image_four_class_classification_model_4[images.split('_')[-1].split('.')[0]+'_image'] = my_predictions_4
            
            
    #print(my_predictions_study)
    if len(my_predictions_study) > 1:
        my_predictions_study = list(my_predictions_study[1,:])


        prediction_string = 'negative' + ' ' + str(my_predictions_study[0]) + ' 0 0 1 1 ' + 'typical' + ' ' + str(my_predictions_study[1]) + ' 0 0 1 1 ' + 'indeterminate' + ' ' + str(my_predictions_study[2]) + ' 0 0 1 1 ' + 'atypical' + ' ' + str(my_predictions_study[3]) + ' 0 0 1 1' 

        #print(prediction_string)

        search_name = study_name[0] + '_study'
        my_submission_file_4_study['PredictionString'][my_submission_file_4_study.id==search_name] = prediction_string

In [ ]:
del my_model_4
gc.collect()

my_submission_file_4_study

In [ ]:
my_model_5 = tf.keras.models.load_model('../input/d/asadmkhan/siim-classification-models/model4_ver15.h5')
my_model_5 = tf.keras.Model(inputs=my_model_5.input, outputs=my_model_5.get_layer('Classification_Head').output)
print('Model 5 Loaded')

study_level = my_submission_file_5_study[my_submission_file_5_study['id'].str.contains('_study')]
print(study_level.head())
print(len(study_level))

In [ ]:
for file in tqdm(range(len(study_level))):
    my_row = study_level.loc[file].values.tolist()
    study_name = my_row[0].split('_')

    complete_name = [x for x in all_files if study_name[0] in x]
    
    my_predictions_study = np.zeros((1,4))
    for images in complete_name:
        #print(images)
        
        image_data = cv2.imread(os.path.join(rgb_test_images_path, images), -1)
        if image_data is not None:
            my_image_5 = cv2.resize(image_data, image_size_5[:2])/255.0
            my_image_expanded_5 = np.zeros((3,*image_size_5))
            my_image_expanded_5[0,...] = my_image_5
            my_image_expanded_5[1,...] = np.fliplr(my_image_5)
            my_image_expanded_5[2,...] = np.roll(my_image_5, 30, axis=1)
            
            my_predictions_5 = my_model_5.predict(my_image_expanded_5)
            
            #try:
            #    index = np.argmax(my_predictions_5[:,statistics.mode(np.argmax(my_predictions_5, axis=0))],axis=0)
            #    my_predictions_5 = np.reshape(my_predictions_5[index,:], (1,4))
            #except:
            my_predictions_5 = np.reshape(np.mean(my_predictions_5, axis=0), (1,4))
            
            del my_image_5
            del my_image_expanded_5
            
            my_predictions_study = np.concatenate((my_predictions_study, my_predictions_5))
            
            #my_prediction_image = np.mean(np.vstack((my_predictions_1)), axis=0)
            image_four_class_classification_model_5[images.split('_')[-1].split('.')[0]+'_image'] = my_predictions_5
            
            
    #print(my_predictions_study)
    if len(my_predictions_study) > 1:
        my_predictions_study = list(my_predictions_study[1,:])


        prediction_string = 'negative' + ' ' + str(my_predictions_study[0]) + ' 0 0 1 1 ' + 'typical' + ' ' + str(my_predictions_study[1]) + ' 0 0 1 1 ' + 'indeterminate' + ' ' + str(my_predictions_study[2]) + ' 0 0 1 1 ' + 'atypical' + ' ' + str(my_predictions_study[3]) + ' 0 0 1 1' 

        #print(prediction_string)

        search_name = study_name[0] + '_study'
        my_submission_file_5_study['PredictionString'][my_submission_file_5_study.id==search_name] = prediction_string

In [ ]:
del my_model_5
gc.collect()

my_submission_file_4_study

In [ ]:
gc.collect()

In [ ]:
my_submission_file_6_study = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
my_submission_file_7_study = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
my_submission_file_8_study = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
my_submission_file_9_study = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
my_submission_file_10_study = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')

image_size_6 = (768, 768, 3)
image_size_7 = (768, 768, 3)
image_size_8 = (768, 768, 3)
image_size_9 = (768, 768, 3)
image_size_10 = (768, 768, 3)

In [ ]:
my_model_6 = tf.keras.models.load_model('../input/siim-effnetv2-keras-study-train-tpu-cv0-805/model0_ver16.h5')
my_model_6 = tf.keras.Model(inputs=my_model_6.input, outputs=my_model_6.get_layer('Classification_Head').output)
print('Model 6 Loaded')

study_level = my_submission_file_6_study[my_submission_file_6_study['id'].str.contains('_study')]
print(study_level.head())
print(len(study_level))

In [ ]:
for file in tqdm(range(len(study_level))):
    my_row = study_level.loc[file].values.tolist()
    study_name = my_row[0].split('_')

    complete_name = [x for x in all_files if study_name[0] in x]
    
    my_predictions_study = np.zeros((1,4))
    for images in complete_name:
        #print(images)
        
        image_data = cv2.imread(os.path.join(rgb_test_images_path, images), -1)
        if image_data is not None:
            my_image_6 = cv2.resize(image_data, image_size_6[:2])/255.0
            my_image_expanded_6 = np.zeros((3,*image_size_6))
            my_image_expanded_6[0,...] = my_image_6
            my_image_expanded_6[1,...] = np.fliplr(my_image_6)
            my_image_expanded_6[2,...] = np.roll(my_image_6, 30, axis=1)
            
            my_predictions_6 = my_model_6.predict(my_image_expanded_6)
            
            #try:
            #    index = np.argmax(my_predictions_6[:,statistics.mode(np.argmax(my_predictions_6, axis=0))],axis=0)
            #    my_predictions_6 = np.reshape(my_predictions_6[index,:], (1,4))
            #except:
            my_predictions_6 = np.reshape(np.mean(my_predictions_6, axis=0), (1,4))
            
            del my_image_6
            del my_image_expanded_6
            
            my_predictions_study = np.concatenate((my_predictions_study, my_predictions_6))
            
            #my_prediction_image = np.mean(np.vstack((my_predictions_1)), axis=0)
            image_four_class_classification_model_6[images.split('_')[-1].split('.')[0]+'_image'] = my_predictions_6
            
            
    #print(my_predictions_study)
    if len(my_predictions_study) > 1:
        my_predictions_study = list(my_predictions_study[1,:])


        prediction_string = 'negative' + ' ' + str(my_predictions_study[0]) + ' 0 0 1 1 ' + 'typical' + ' ' + str(my_predictions_study[1]) + ' 0 0 1 1 ' + 'indeterminate' + ' ' + str(my_predictions_study[2]) + ' 0 0 1 1 ' + 'atypical' + ' ' + str(my_predictions_study[3]) + ' 0 0 1 1' 

        #print(prediction_string)

        search_name = study_name[0] + '_study'
        my_submission_file_6_study['PredictionString'][my_submission_file_6_study.id==search_name] = prediction_string

In [ ]:
del my_model_6
gc.collect()
my_submission_file_6_study

In [ ]:
my_model_7 = tf.keras.models.load_model('../input/siim-effnetv2-keras-study-train-tpu-cv0-805/model1_ver16.h5')
my_model_7 = tf.keras.Model(inputs=my_model_7.input, outputs=my_model_7.get_layer('Classification_Head').output)
print('Model 7 Loaded')

study_level = my_submission_file_7_study[my_submission_file_7_study['id'].str.contains('_study')]
print(study_level.head())
print(len(study_level))

In [ ]:
for file in tqdm(range(len(study_level))):
    my_row = study_level.loc[file].values.tolist()
    study_name = my_row[0].split('_')

    complete_name = [x for x in all_files if study_name[0] in x]
    
    my_predictions_study = np.zeros((1,4))
    for images in complete_name:
        #print(images)
        
        image_data = cv2.imread(os.path.join(rgb_test_images_path, images), -1)
        if image_data is not None:
            my_image_7 = cv2.resize(image_data, image_size_7[:2])/255.0
            my_image_expanded_7 = np.zeros((3,*image_size_7))
            my_image_expanded_7[0,...] = my_image_7
            my_image_expanded_7[1,...] = np.fliplr(my_image_7)
            my_image_expanded_7[2,...] = np.roll(my_image_7, 30, axis=1)
            
            my_predictions_7 = my_model_7.predict(my_image_expanded_7)
            
            #try:
            #    index = np.argmax(my_predictions_7[:,statistics.mode(np.argmax(my_predictions_7, axis=0))],axis=0)
            #    my_predictions_7 = np.reshape(my_predictions_7[index,:], (1,4))
            #except:
            my_predictions_7 = np.reshape(np.mean(my_predictions_7, axis=0), (1,4))
            
            
            del my_image_7
            del my_image_expanded_7
            
            my_predictions_study = np.concatenate((my_predictions_study, my_predictions_7))
            
            #my_prediction_image = np.mean(np.vstack((my_predictions_1)), axis=0)
            image_four_class_classification_model_7[images.split('_')[-1].split('.')[0]+'_image'] = my_predictions_7
            
            
    if len(my_predictions_study) > 1:
        my_predictions_study = list(my_predictions_study[1,:])


        prediction_string = 'negative' + ' ' + str(my_predictions_study[0]) + ' 0 0 1 1 ' + 'typical' + ' ' + str(my_predictions_study[1]) + ' 0 0 1 1 ' + 'indeterminate' + ' ' + str(my_predictions_study[2]) + ' 0 0 1 1 ' + 'atypical' + ' ' + str(my_predictions_study[3]) + ' 0 0 1 1' 

        #print(prediction_string)

        search_name = study_name[0] + '_study'
        my_submission_file_7_study['PredictionString'][my_submission_file_7_study.id==search_name] = prediction_string

In [ ]:
del my_model_7
gc.collect()

my_submission_file_7_study

In [ ]:
my_model_8 = tf.keras.models.load_model('../input/siim-effnetv2-keras-study-train-tpu-cv0-805/model2_ver16.h5')
my_model_8 = tf.keras.Model(inputs=my_model_8.input, outputs=my_model_8.get_layer('Classification_Head').output)
print('Model 8 Loaded')

study_level = my_submission_file_8_study[my_submission_file_8_study['id'].str.contains('_study')]
print(study_level.head())
print(len(study_level))

In [ ]:
for file in tqdm(range(len(study_level))):
    my_row = study_level.loc[file].values.tolist()
    study_name = my_row[0].split('_')

    complete_name = [x for x in all_files if study_name[0] in x]
    
    my_predictions_study = np.zeros((1,4))
    for images in complete_name:
        
        image_data = cv2.imread(os.path.join(rgb_test_images_path, images), -1)
        if image_data is not None:
            my_image_8 = cv2.resize(image_data, image_size_8[:2])/255.0
            my_image_expanded_8 = np.zeros((3,*image_size_8))
            my_image_expanded_8[0,...] = my_image_8
            my_image_expanded_8[1,...] = np.fliplr(my_image_8)
            my_image_expanded_8[2,...] = np.roll(my_image_8, 30, axis=1)
            
            my_predictions_8 = my_model_8.predict(my_image_expanded_8)
            
            #try:
            #    index = np.argmax(my_predictions_8[:,statistics.mode(np.argmax(my_predictions_8, axis=0))],axis=0)
            #    my_predictions_8 = np.reshape(my_predictions_8[index,:], (1,4))
            #except:
            my_predictions_8 = np.reshape(np.mean(my_predictions_8, axis=0), (1,4))
            
            del my_image_8
            del my_image_expanded_8
            
            my_predictions_study = np.concatenate((my_predictions_study, my_predictions_8))
            
            #my_prediction_image = np.mean(np.vstack((my_predictions_1)), axis=0)
            image_four_class_classification_model_8[images.split('_')[-1].split('.')[0]+'_image'] = my_predictions_8
            
            
    if len(my_predictions_study) > 1:
        my_predictions_study = list(my_predictions_study[1,:])


        prediction_string = 'negative' + ' ' + str(my_predictions_study[0]) + ' 0 0 1 1 ' + 'typical' + ' ' + str(my_predictions_study[1]) + ' 0 0 1 1 ' + 'indeterminate' + ' ' + str(my_predictions_study[2]) + ' 0 0 1 1 ' + 'atypical' + ' ' + str(my_predictions_study[3]) + ' 0 0 1 1' 

        #print(prediction_string)

        search_name = study_name[0] + '_study'
        my_submission_file_8_study['PredictionString'][my_submission_file_8_study.id==search_name] = prediction_string

In [ ]:
del my_model_8
gc.collect()

my_submission_file_3_study

In [ ]:
my_model_9 = tf.keras.models.load_model('../input/siim-effnetv2-keras-study-train-tpu-cv0-805/model3_ver16.h5')
my_model_9 = tf.keras.Model(inputs=my_model_9.input, outputs=my_model_9.get_layer('Classification_Head').output)
print('Model 9 Loaded')

study_level = my_submission_file_9_study[my_submission_file_9_study['id'].str.contains('_study')]
print(study_level.head())
print(len(study_level))

In [ ]:
for file in tqdm(range(len(study_level))):
    my_row = study_level.loc[file].values.tolist()
    study_name = my_row[0].split('_')

    complete_name = [x for x in all_files if study_name[0] in x]
    
    my_predictions_study = np.zeros((1,4))
    for images in complete_name:
        
        image_data = cv2.imread(os.path.join(rgb_test_images_path, images), -1)
        if image_data is not None:
            my_image_9 = cv2.resize(image_data, image_size_9[:2])/255.0
            my_image_expanded_9 = np.zeros((3,*image_size_9))
            my_image_expanded_9[0,...] = my_image_9
            my_image_expanded_9[1,...] = np.fliplr(my_image_9)
            my_image_expanded_9[2,...] = np.roll(my_image_9, 30, axis=1)
            
            my_predictions_9 = my_model_9.predict(my_image_expanded_9)
            
            #try:
            #    index = np.argmax(my_predictions_9[:,statistics.mode(np.argmax(my_predictions_9, axis=0))],axis=0)
            #    my_predictions_9 = np.reshape(my_predictions_9[index,:], (1,4))
            #except:
            my_predictions_9 = np.reshape(np.mean(my_predictions_9, axis=0), (1,4))

            del my_image_9
            del my_image_expanded_9
            
            my_predictions_study = np.concatenate((my_predictions_study, my_predictions_9))
            
            #my_prediction_image = np.mean(np.vstack((my_predictions_1)), axis=0)
            image_four_class_classification_model_9[images.split('_')[-1].split('.')[0]+'_image'] = my_predictions_9
            
            
    if len(my_predictions_study) > 1:
        my_predictions_study = list(my_predictions_study[1,:])


        prediction_string = 'negative' + ' ' + str(my_predictions_study[0]) + ' 0 0 1 1 ' + 'typical' + ' ' + str(my_predictions_study[1]) + ' 0 0 1 1 ' + 'indeterminate' + ' ' + str(my_predictions_study[2]) + ' 0 0 1 1 ' + 'atypical' + ' ' + str(my_predictions_study[3]) + ' 0 0 1 1' 

        #print(prediction_string)

        search_name = study_name[0] + '_study'
        my_submission_file_9_study['PredictionString'][my_submission_file_9_study.id==search_name] = prediction_string

In [ ]:
del my_model_9
gc.collect()

my_submission_file_9_study

In [ ]:
my_model_10 = tf.keras.models.load_model('../input/siim-effnetv2-keras-study-train-tpu-cv0-805/model4_ver16.h5')
my_model_10 = tf.keras.Model(inputs=my_model_10.input, outputs=my_model_10.get_layer('Classification_Head').output)
print('Model 10 Loaded')

study_level = my_submission_file_10_study[my_submission_file_10_study['id'].str.contains('_study')]
print(study_level.head())
print(len(study_level))

In [ ]:
for file in tqdm(range(len(study_level))):
    my_row = study_level.loc[file].values.tolist()
    study_name = my_row[0].split('_')

    complete_name = [x for x in all_files if study_name[0] in x]
    
    my_predictions_study = np.zeros((1,4))
    for images in complete_name:
        
        image_data = cv2.imread(os.path.join(rgb_test_images_path, images), -1)
        if image_data is not None:
            my_image_10 = cv2.resize(image_data, image_size_10[:2])/255.0
            my_image_expanded_10 = np.zeros((3,*image_size_10))
            my_image_expanded_10[0,...] = my_image_10
            my_image_expanded_10[1,...] = np.fliplr(my_image_10)
            my_image_expanded_10[2,...] = np.roll(my_image_10, 30, axis=1)
            
            my_predictions_10 = my_model_10.predict(my_image_expanded_10)
            
            #try:
            #    index = np.argmax(my_predictions_10[:,statistics.mode(np.argmax(my_predictions_10, axis=0))],axis=0)
            #    my_predictions_10 = np.reshape(my_predictions_10[index,:], (1,4))
            #except:
            my_predictions_10 = np.reshape(np.mean(my_predictions_10, axis=0), (1,4))
            
            del my_image_10
            del my_image_expanded_10
            
            my_predictions_study = np.concatenate((my_predictions_study, my_predictions_10))
            
            #my_prediction_image = np.mean(np.vstack((my_predictions_1)), axis=0)
            image_four_class_classification_model_10[images.split('_')[-1].split('.')[0]+'_image'] = my_predictions_10
            
            
    if len(my_predictions_study) > 1:
        my_predictions_study = list(my_predictions_study[1,:])


        prediction_string = 'negative' + ' ' + str(my_predictions_study[0]) + ' 0 0 1 1 ' + 'typical' + ' ' + str(my_predictions_study[1]) + ' 0 0 1 1 ' + 'indeterminate' + ' ' + str(my_predictions_study[2]) + ' 0 0 1 1 ' + 'atypical' + ' ' + str(my_predictions_study[3]) + ' 0 0 1 1' 

        #print(prediction_string)

        search_name = study_name[0] + '_study'
        my_submission_file_10_study['PredictionString'][my_submission_file_10_study.id==search_name] = prediction_string

In [ ]:
del my_model_10
gc.collect()

my_submission_file_10_study

In [ ]:
model_predictions_files = [my_submission_file_1_study, my_submission_file_2_study, my_submission_file_3_study, my_submission_file_4_study, my_submission_file_5_study,
                          my_submission_file_6_study, my_submission_file_7_study, my_submission_file_8_study, my_submission_file_9_study, my_submission_file_10_study]
#model_predictions_files = [my_submission_file_1_study]
print(len(model_predictions_files))

In [ ]:
my_submission_file = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')

In [ ]:
for i in tqdm(range(len(my_submission_file))):
    my_row = my_submission_file.iloc[i]
    if '_study' in my_row.id:
        temp = np.zeros((1,4))
        my_preidction_study = None
        for model_prediction in model_predictions_files:
            classification = model_prediction.iloc[i].PredictionString
            #print(classification)
            classification_parts = classification.split(' ')
            #print(classification_parts)
            if len(classification_parts) > 6:
                temp = np.vstack((temp, np.array([float(classification_parts[1]), float(classification_parts[7]), float(classification_parts[13]), float(classification_parts[19])])))
            else:
                temp = np.vstack((temp, np.array([float(classification_parts[1]), 0, 0, 0])))
                
        #print(temp)
        #my_predictions_study = list(np.mean(temp[1:11,:], axis=0))
        my_predictions_study = list(np.average(temp[1:11,:], axis=0, weights=[0.11061947, 0.14159292, 0.13274336, 0.10176991, 0.04424779, 0.04424779, 0.11061947, 0.10619469, 0.10619469, 0.10176991]))
        
        prediction_string = 'negative' + ' ' + str(my_predictions_study[0]) + ' 0 0 1 1 ' + 'typical' + ' ' + str(my_predictions_study[1]) + ' 0 0 1 1 ' + 'indeterminate' + ' ' + str(my_predictions_study[2]) + ' 0 0 1 1 ' + 'atypical' + ' ' + str(my_predictions_study[3]) + ' 0 0 1 1' 
        
    
        my_submission_file['PredictionString'][my_submission_file.id==my_row.id] = prediction_string

In [ ]:
my_submission_file

# **Image Level Localisation**

In [ ]:
my_submission_file_1_image = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
my_submission_file_2_image = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
my_submission_file_3_image = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
my_submission_file_4_image = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
my_submission_file_5_image = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
my_submission_file_6_image = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')

In [ ]:
print(len(os.listdir('./test_images_rgb/')))
print(len(os.listdir('./test_images_psuedo_color/')))

In [ ]:
!python ../input/yolov5/yolov5/yolov5/detect.py --weights '../input/yolov5/weights_1/best.pt' --source './test_images_psuedo_color/' --img 640 --conf 0.001 --iou-thres 0.5  --save-txt --save-conf --nosave --project './model_1_outputs/'  --augment

In [ ]:
annotation_files = os.listdir(os.path.join(model_1_outputs_path, 'exp', 'labels'))
annotation_files = [x for x in annotation_files if '.txt' in x]
print(len(annotation_files))

In [ ]:
classes = ['lung opacity']

image_pred_strings = []
for file in tqdm(annotation_files):
    
    search_name = file.split('_')[-1].split('.')[0] + '_image'
    confidence, bboxes = get_conf_bboxes(os.path.join(model_1_outputs_path, 'exp', 'labels', file))
    #print(image_height_width[search_name][0],image_height_width[search_name][1])
    bboxes = correct_bbox_format(bboxes,image_height_width[search_name][0],image_height_width[search_name][1])
    
    prediction_string = ''
    
    for j, conf in enumerate(confidence):
        prediction_string += f'opacity {conf} ' + ' '.join(map(str, bboxes[j])) + ' '        
    
    prediction_string = prediction_string[:-1] 
    
    
    #print(prediction_string)

    prediction_string = bounding_box_sorter(search_name,prediction_string)
    my_submission_file_1_image['PredictionString'][my_submission_file_1_image.id==search_name] = prediction_string

In [ ]:
!python ../input/yolov5/yolov5/yolov5/detect.py --weights '../input/yolov5/weights_2/best.pt' --source './test_images_psuedo_color/' --img 1088 --conf 0.001 --iou-thres 0.5  --save-txt --save-conf --nosave --project './model_2_outputs/' --augment

In [ ]:
annotation_files = os.listdir(os.path.join(model_2_outputs_path, 'exp', 'labels'))
annotation_files = [x for x in annotation_files if '.txt' in x]
print(len(annotation_files))
#print(annotation_files)

In [ ]:
classes = ['lung opacity']

image_pred_strings = []
for file in tqdm(annotation_files):
    
    search_name = file.split('_')[-1].split('.')[0] + '_image'
    confidence, bboxes = get_conf_bboxes(os.path.join(model_2_outputs_path, 'exp', 'labels', file))
    #print(image_height_width[search_name][0],image_height_width[search_name][1])
    bboxes = correct_bbox_format(bboxes, image_height_width[search_name][0],image_height_width[search_name][1])
    
    prediction_string = ''
    
    for j, conf in enumerate(confidence):
        prediction_string += f'opacity {conf} ' + ' '.join(map(str, bboxes[j])) + ' '        
    
    prediction_string = prediction_string[:-1] 
    
    
    #print(prediction_string)

    prediction_string = bounding_box_sorter(search_name,prediction_string)
    my_submission_file_2_image['PredictionString'][my_submission_file_2_image.id==search_name] = prediction_string

In [ ]:
!python ../input/yolov5/yolov5/yolov5/detect.py --weights '../input/yolov5/weights_3/best.pt' --source './test_images_rgb/' --img 640 --conf 0.001 --iou-thres 0.5  --save-txt --save-conf --nosave --project './model_3_outputs/'  --augment

In [ ]:
annotation_files = os.listdir(os.path.join(model_3_outputs_path, 'exp', 'labels'))
annotation_files = [x for x in annotation_files if '.txt' in x]
print(len(annotation_files))
#print(annotation_files)

In [ ]:
classes = ['lung opacity']

image_pred_strings = []
for file in tqdm(annotation_files):
    
    search_name = file.split('_')[-1].split('.')[0] + '_image'
    confidence, bboxes = get_conf_bboxes(os.path.join(model_3_outputs_path, 'exp', 'labels', file))
    #print(image_height_width[search_name][0],image_height_width[search_name][1])
    bboxes = correct_bbox_format(bboxes, image_height_width[search_name][0],image_height_width[search_name][1])
    
    prediction_string = ''
    
    for j, conf in enumerate(confidence):
        prediction_string += f'opacity {conf} ' + ' '.join(map(str, bboxes[j])) + ' '        
    
    prediction_string = prediction_string[:-1] 
    
    
    #print(prediction_string)

    prediction_string = bounding_box_sorter(search_name,prediction_string)
    my_submission_file_3_image['PredictionString'][my_submission_file_3_image.id==search_name] = prediction_string

In [ ]:
!python ../input/yolov5/yolov5/yolov5/detect.py --weights '../input/yolov5/weights_4/best.pt' --source './test_images_psuedo_color/' --img 640 --conf 0.001 --iou-thres 0.5  --save-txt --save-conf --nosave --project './model_4_outputs/' --augment

In [ ]:
annotation_files = os.listdir(os.path.join(model_4_outputs_path, 'exp', 'labels'))
annotation_files = [x for x in annotation_files if '.txt' in x]
print(len(annotation_files))
#print(annotation_files)

In [ ]:
classes = ['lung opacity']

image_pred_strings = []
for file in tqdm(annotation_files):
    
    search_name = file.split('_')[-1].split('.')[0] + '_image'
    confidence, bboxes = get_conf_bboxes(os.path.join(model_4_outputs_path, 'exp', 'labels', file))
    #print(image_height_width[search_name][0],image_height_width[search_name][1])
    bboxes = correct_bbox_format(bboxes, image_height_width[search_name][0],image_height_width[search_name][1])
    
    prediction_string = ''
    
    for j, conf in enumerate(confidence):
        prediction_string += f'opacity {conf} ' + ' '.join(map(str, bboxes[j])) + ' '        
    
    prediction_string = prediction_string[:-1] 
    
    
    #print(prediction_string)

    prediction_string = bounding_box_sorter(search_name,prediction_string)
    my_submission_file_4_image['PredictionString'][my_submission_file_4_image.id==search_name] = prediction_string

This one is to be ignored****

In [ ]:
!python ../input/yolov5/yolov5/yolov5/detect.py --weights '../input/yolov5/weights_9/best.pt' --source './test_images_psuedo_color/' --img 1088 --conf 0.001 --iou-thres 0.5  --save-txt --save-conf --nosave --project './model_5_outputs/'  --augment

In [ ]:
annotation_files = os.listdir(os.path.join(model_5_outputs_path, 'exp', 'labels'))
annotation_files = [x for x in annotation_files if '.txt' in x]
print(len(annotation_files))
#print(annotation_files)

In [ ]:
classes = ['lung opacity']

image_pred_strings = []
for file in tqdm(annotation_files):
    
    search_name = file.split('_')[-1].split('.')[0] + '_image'
    confidence, bboxes = get_conf_bboxes(os.path.join(model_5_outputs_path, 'exp', 'labels', file))
    #print(image_height_width[search_name][0],image_height_width[search_name][1])
    bboxes = correct_bbox_format(bboxes, image_height_width[search_name][0],image_height_width[search_name][1])
    
    prediction_string = ''
    
    for j, conf in enumerate(confidence):
        prediction_string += f'opacity {conf} ' + ' '.join(map(str, bboxes[j])) + ' '        
    
    prediction_string = prediction_string[:-1] 
    
    
    #print(prediction_string)

    prediction_string = bounding_box_sorter(search_name,prediction_string)
    my_submission_file_5_image['PredictionString'][my_submission_file_5_image.id==search_name] = prediction_string

In [ ]:
!python ../input/yolov5/yolov5/yolov5/detect.py --weights '../input/yolov5/best.pt' --source './test_images_rgb/' --img 512 --conf 0.001 --iou-thres 0.5  --save-txt --save-conf --nosave --project './model_6_outputs/'  --augment

In [ ]:
annotation_files = os.listdir(os.path.join(model_6_outputs_path, 'exp', 'labels'))
annotation_files = [x for x in annotation_files if '.txt' in x]
print(len(annotation_files))
#print(annotation_files)

In [ ]:
classes = ['lung opacity']

image_pred_strings = []
for file in tqdm(annotation_files):
    
    search_name = file.split('_')[-1].split('.')[0] + '_image'
    confidence, bboxes = get_conf_bboxes(os.path.join(model_6_outputs_path, 'exp', 'labels', file))
    #print(image_height_width[search_name][0],image_height_width[search_name][1])
    bboxes = correct_bbox_format(bboxes, image_height_width[search_name][0],image_height_width[search_name][1])
    
    prediction_string = ''
    
    for j, conf in enumerate(confidence):
        prediction_string += f'opacity {conf} ' + ' '.join(map(str, bboxes[j])) + ' '        
    
    prediction_string = prediction_string[:-1] 
    
    
    #print(prediction_string)

    prediction_string = bounding_box_sorter(search_name,prediction_string)
    my_submission_file_6_image['PredictionString'][my_submission_file_6_image.id==search_name] = prediction_string

In [ ]:
models_predictions_file = [my_submission_file_1_image, my_submission_file_2_image, my_submission_file_3_image, my_submission_file_4_image, my_submission_file_5_image, my_submission_file_6_image]
#models_predictions_file = [my_submission_file_1_image]
my_keys = list(image_height_width.keys())

for i in tqdm(range(len(my_submission_file))):
    my_row = my_submission_file.iloc[i]
    if '_image' in my_row.id:
        boxes_list = []
        scores_list = []
        labels_list = []
        weights = []
        boxes = []
        iou_thr = 0.55
        skip_box_thr = 0.0013
        sigma = 0.1
        if my_row.id in my_keys:
            image_height, image_width = image_height_width[my_row.id][0],image_height_width[my_row.id][1]
            how_many_predictions = 0
            for counter, model in enumerate(models_predictions_file):
                current_model_predictions = model['PredictionString'][model.id==my_row.id].values
                current_model_predictions = current_model_predictions[0].split(' ')
                #print(counter)
                #print(current_model_predictions)
                if 'none' not in current_model_predictions:
                    how_many_predictions +=1
                    temp_boxes = []
                    temp_scores = []
                    temp_labels = []

                    for i in range(0,len(current_model_predictions), 6):
                        current_annotation = current_model_predictions[i:i+6]
                        temp_scores.append(float(current_annotation[1]))
                        temp_labels.append(0)
                        bounding_box = [float(x) for x in current_annotation[2:]]
                        #print(bounding_box)
                        bounding_box[0], bounding_box[1], bounding_box[2], bounding_box[3] = (bounding_box[0]-1)/image_width, (bounding_box[1]-1)/image_height, (bounding_box[2]-1)/image_width, (bounding_box[3]-1)/image_height
                        temp_boxes.append(bounding_box)

                    boxes_list.append(temp_boxes)
                    scores_list.append(temp_scores)
                    labels_list.append(temp_labels)



            if how_many_predictions == 1:
                weights = [1]
                boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
                boxes = denormalise(boxes, image_height, image_width)

            elif how_many_predictions >1:
                weights = [1, 1, 1, 1, 1, 1.25]
                boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
                boxes = denormalise(boxes, image_height, image_width)



            if len(boxes)>0:

                prediction_string = ''

                for i in range(len(boxes)):
                    prediction_string += 'opacity ' + str(scores[i]) + ' ' + str(int(boxes[i][0])) + ' ' + str(int(boxes[i][1])) + ' ' + str(int(boxes[i][2])) + ' '  + str(int(boxes[i][3])) + ' '

                prediction_string=prediction_string[:-1]
                #print(prediction_string)

                prediction_string = bounding_box_sorter(my_row.id,prediction_string)
                #prediction_string = keep_n_top_loaclisations(prediction_string, 49)
                my_submission_file['PredictionString'][my_submission_file.id==my_row.id] = prediction_string

In [ ]:
my_submission_file

# 2-Class Classification

In [ ]:
model_predictions_files = [image_four_class_classification_model_1, image_four_class_classification_model_2, image_four_class_classification_model_3, image_four_class_classification_model_4, image_four_class_classification_model_5,
                            image_four_class_classification_model_6, image_four_class_classification_model_7, image_four_class_classification_model_8, image_four_class_classification_model_9, image_four_class_classification_model_10]

#model_predictions_files = [image_four_class_classification_model_1, image_four_class_classification_model_2, image_four_class_classification_model_3,
#                           image_four_class_classification_model_4, image_four_class_classification_model_5]

#model_predictions_files = [my_submission_file_1_study]
print(len(model_predictions_files))

In [ ]:
all_keys = image_four_class_classification_model_1.keys()
for i in tqdm(range(len(my_submission_file))):
    my_row = my_submission_file.iloc[i]
    if '_image' in my_row.id:
        if my_row.id in all_keys:
            search_name = [x for x in all_keys if my_row.id in x]

            temp = np.zeros((1,4))
            for model_predictions in model_predictions_files:
                temp = np.vstack((temp, model_predictions[search_name[0]]))

            #my_predictions_combined = np.mean(temp, axis=0)[0]
            my_predictions_combined = np.average(temp[1:11,:], axis=0, weights=[0.11061947, 0.14159292, 0.13274336, 0.10176991, 0.04424779, 0.04424779, 0.11061947, 0.10619469, 0.10619469, 0.10176991])[0]
            #print(my_predictions_combined)

            current_annotation = my_submission_file['PredictionString'][my_submission_file.id==search_name[0]].values[0]
            if 'none' in current_annotation:
                prediction_string = 'none ' + str(my_predictions_combined) + ' 0 0 1 1'
                my_submission_file['PredictionString'][my_submission_file.id==search_name[0]] = prediction_string
            else:
                current_annotation += ' none ' + str(my_predictions_combined) + ' 0 0 1 1'
                my_submission_file['PredictionString'][my_submission_file.id==search_name[0]] = current_annotation

In [ ]:
my_submission_file

# Removing Some Particular Classification/Localisation

In [ ]:
for i in tqdm(range(len(my_submission_file))):
    my_row = my_submission_file.iloc[i]
    if '_study' in my_row.id:
        my_submission_file['PredictionString'][my_submission_file.id==my_row.id] = 'negative 1 0 0 1 1'
        

In [ ]:
my_submission_file

In [ ]:
my_submission_file.to_csv(os.path.join('./', 'submission.csv'), index=False, sep=',')

In [ ]:
!rm -rf ./test_images_psuedo_color
!rm -rf ./test_images_rgb
!rm -rf ./model_1_outputs
!rm -rf ./model_2_outputs
!rm -rf ./model_3_outputs
!rm -rf ./model_4_outputs
!rm -rf ./model_5_outputs
!rm -rf ./model_6_outputs